# AI-generated CV testing

## Data retreival

In [3]:
import pandas as pd
import psycopg2

In [18]:
with psycopg2.connect(
    dbname="cv_manager",
    user="cvuser",
    password="cvpassword",
    host="localhost",
    port='5432'
) as conn:
    query = "SELECT * FROM experiences;"
    data = pd.read_sql(query, conn)

/var/folders/qj/506gdrfn55jc0hchzk5d05540000gn/T/ipykernel_23856/192697586.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


Let's create a CV for user 1. 

In [57]:
# let's create a CV for user 1
query = """
SELECT 
    e.organisation,
    e.title as engagement_title,
    ex.type,
    ex.title as experience_title,
    ex.description
FROM 
    users u
LEFT JOIN engagements e on u.user_id = e.user_id
LEFT JOIN experiences ex on e.engagement_id = ex.engagement_id
where u.user_id=1;
"""
with psycopg2.connect(
    dbname="cv_manager",
    user="cvuser",
    password="cvpassword",
    host="localhost",
    port='5432'
) as conn:
    data = pd.read_sql(query, conn)

data

/var/folders/qj/506gdrfn55jc0hchzk5d05540000gn/T/ipykernel_23856/2523801472.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


,organisation,engagement_title,type,experience_title,description
0,Google,Software Engineer,work,Software Engineer,Situation: Google was scaling its cloud servic...
1,Google,Software Engineer,work,Software Engineer,Situation: Google needed a secure authenticati...
2,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: High customer volume at McDonald's....
3,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: Frequent errors in order fulfillmen...
4,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Developed custom web applications f...
5,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Clients needed better data insights...
6,MIT,Computer Science BSc,education,Computer Science BSc,Situation: Studying foundational computer scie...


# CV generation

Let's create a crew of agents who can generate a CV. First, let's build an agent which can, when given a job description, identify the most relevant experiences from a master CV.

## Agent 1 - choosing experience most relevant to the job description

In [244]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio",
    verbose=True
)

In [245]:
role = "Senior Software Engineer"

job_requirements = """
What it's all about…

As a Engineering Tech Lead, you will be responsible for:

Define and communicate technical strategies aligning with business strategies
Identify, and evaluate technical solutions to drive product and ecommerce vision
Identify and implement new technologies, tools and processes to enhance efficiency and product quality
Design and develop scalable and efficient systems
Optimise system performance, enhance security and maintain high-quality codebases
Implement and manage CI/CD pipelines for streamlined software development
Conduct code reviews, establish coding standards and best practices
Lead, mentor and inspire a team of engineers
Encourage collaboration and continuous improvement
Represent the engineering team and contribute to planning and decision making
Ensure seamless integration across frontend and backend components
Collaborate on database scaling and optimisation using technologies such as MySQL and PlanetScale
Work closely with the Product Manager to define and prioritise the roadmap ensuring technical deliverables
What we will need from you…

Previous experience working in a frontend and backend software engineering role
Proven experience in leading a technical team and mentoring engineers
Strong proficiency in JavaScript, TypeScript and modern web frameworks (React, Node.js)
Solid understanding of web service architectures (REST, GraphQL)
Experience with web analytics platforms (GA4, Google Tag Manager) and tracking strategies
Experience with cloud-based infrastructure (Fly.io, AWS) and container technologies (Docker)
Experience with Agile methodologies and collaborative tools (Notion, JIRA, Trello)
Demonstrated success in delivering and optimising digital products
Knowledge in testing strategies, performance optimisation and DevOps principles
A passion for continuous learning and driving technical innovation
Excellent problem-solving skills and attention to detail
Strong communication skills
"""

In [246]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a professional career coach. You will be given a master CV and details of the role which the user is applying for.
Your job is to identify the top 3 most relevant work experiences and explain why. 
"""

user_prompt = """
Role: {role}

Job Requirements: {job_requirements}

Master CV: {master_cv}

### Output format:
Ensure `choices`, `reasoning` and `selected_experiences` are **valid Python lists**, not strings of lists.
"""


prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_prompt),
     ("user", user_prompt)]
)

In [247]:
prompt = prompt_template.invoke({"role": role, "job_requirements": job_requirements, "master_cv": data})
print(prompt.to_messages()[1].content)


Role: Senior Software Engineer

Job Requirements: 
What it's all about…

As a Engineering Tech Lead, you will be responsible for:

Define and communicate technical strategies aligning with business strategies
Identify, and evaluate technical solutions to drive product and ecommerce vision
Identify and implement new technologies, tools and processes to enhance efficiency and product quality
Design and develop scalable and efficient systems
Optimise system performance, enhance security and maintain high-quality codebases
Implement and manage CI/CD pipelines for streamlined software development
Conduct code reviews, establish coding standards and best practices
Lead, mentor and inspire a team of engineers
Encourage collaboration and continuous improvement
Represent the engineering team and contribute to planning and decision making
Ensure seamless integration across frontend and backend components
Collaborate on database scaling and optimisation using technologies such as MySQL and Plane

In [248]:
from pydantic import BaseModel, Field, field_validator
from typing import List
import json

# Pydantic
class ExperienceSelection(BaseModel):
    """Structured selection of work experiences for job matching."""

    choices: List[int] = Field(description="Indexes of the top 3 selected experiences from the Master CV, in order of relevance.")
    selected_experiences: List[str] = Field(description="The actual descriptions of the selected experiences")
    reasons: List[str] = Field(description="A reason for choosing each experience.")

    @field_validator("choices", mode="before")
    @classmethod
    def parse_list_of_integers(cls, value):
        """Ensure choices is always a list of integers"""
        if isinstance(value, str):
            try:
                return json.loads(value)  # Convert string to list
            except json.JSONDecodeError:
                raise ValueError(f"Invalid list format: {value}")
        return value  # Already a list, return as is
    
    @field_validator("selected_experiences", "reasons", mode="before")
    @classmethod
    def parse_list_of_strings(cls, value):
        """Ensure selected_experiences is always a list of strings."""
        if isinstance(value, str):
            try:
                return json.loads(value)  # Convert string to list
            except json.JSONDecodeError:
                raise ValueError(f"Invalid list format: {value}")
        return value  # Already a list, return as is


structured_llm = llm.with_structured_output(ExperienceSelection)

chain = prompt_template | structured_llm

/opt/anaconda3/envs/cv_env/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1382: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [249]:
response = chain.invoke({"role": role, "job_requirements": job_requirements, "master_cv": data})

In [250]:
data

,organisation,engagement_title,type,experience_title,description
0,Google,Software Engineer,work,Software Engineer,Situation: Google was scaling its cloud servic...
1,Google,Software Engineer,work,Software Engineer,Situation: Google needed a secure authenticati...
2,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: High customer volume at McDonald's....
3,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: Frequent errors in order fulfillmen...
4,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Developed custom web applications f...
5,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Clients needed better data insights...
6,MIT,Computer Science BSc,education,Computer Science BSc,Situation: Studying foundational computer scie...


In [251]:
data.iloc[response.choices, :]

,organisation,engagement_title,type,experience_title,description
0,Google,Software Engineer,work,Software Engineer,Situation: Google was scaling its cloud servic...
4,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Developed custom web applications f...
5,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Clients needed better data insights...


## Agent 2 - rewriting experiences into bullet points

In [236]:
system_prompt2 = """
You are a professional CV writer. Rewrite the relevant experience below as a short bullet point. Always include the action and result from the experience. Do not make anything up. Ensure your output is the bullet point only and ready to be used in a CV.

"""

user_prompt2 = """
User's relevant experience: 
{relevant_experience}
"""


prompt_template2 = ChatPromptTemplate.from_messages(
    [("system", system_prompt2),
     ("user", user_prompt2)]
)

In [237]:
# Pydantic
class BulletPoints(BaseModel):
    """Structured bullet points of work experiences for job matching."""

    bullet_points: List[str] = Field(description="Bullet point for each of the relevant experience provided, written in past tense")

    @field_validator("bullet_points", mode="before")
    @classmethod
    def parse_list_of_strings(cls, value):
        """Ensure bullet_points is always a list of strings."""
        if isinstance(value, str):
            try:
                return json.loads(value)  # Convert string to list
            except json.JSONDecodeError:
                raise ValueError(f"Invalid list format: {value}")
        return value  # Already a list, return as is


structured_llm2 = llm.with_structured_output(BulletPoints)

chain2 = prompt_template2 | llm

/opt/anaconda3/envs/cv_env/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1382: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [238]:
print(prompt_template2.invoke({"relevant_experience": data.iloc[response.choices[0], :]}).to_messages()[1].content)


User's relevant experience: 
organisation                                                   Google
engagement_title                                    Software Engineer
type                                                             work
experience_title                                    Software Engineer
description         Situation: Google was scaling its cloud servic...
Name: 0, dtype: object



In [239]:
print(data.iloc[response.choices[0]][['experience_title', 'description']])

experience_title                                    Software Engineer
description         Situation: Google was scaling its cloud servic...
Name: 0, dtype: object


In [242]:
response2 = chain2.invoke({"relevant_experience": data.iloc[response.choices[1]][['experience_title', 'description']]})

In [243]:
print(response2.content)

• Developed and implemented multiple security protocols for Google's authentication system, resulting in a 99.9% success rate of secure login transactions.
